In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上的端到端机器学习：MLOps阶段6：使用Vertex AI Prediction的TensorFlow模型Serving功能开始

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving_function.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving_function.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving_function.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>         
</table>
<br/><br/><br/>

## 概述

本教程演示了如何将模型部署到 `Vertex AI Endpoint` 并添加服务功能。

### 目标

在本教程中，您将学习如何在具有服务功能的`Vertex AI Endpoint`资源上使用`Vertex AI Prediction`。

本教程使用以下谷歌云ML服务和资源：

- `Vertex AI Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 从TensorFlow Hub下载预训练的图像分类模型。
- 创建一个服务函数来接收压缩的图像数据，并输出预处理后的解压缩数据作为模型输入。
- 将TensorFlow Hub模型和服务函数上传为`Vertex AI Model`资源。
- 创建一个`Endpoint`资源。
- 将`Model`资源部署到`Endpoint`资源上。
- 对部署到`Endpoint`资源的`Model`资源实例进行在线预测。

数据集

本教程使用了 TensorFlow Hub 中一个预训练的图像分类模型，该模型是在 ImageNet 数据集上训练的。

了解更多关于 [ResNet V2 预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您预期的使用情况生成成本估算。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q

### 重新启动内核

在安装额外的包之后，您需要重新启动笔记本的内核，以便它能找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

*如果您有此选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的Google Cloud项目

**以下步骤是必需的，无论您的笔记本环境如何。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您会获得$300的免费信用额度用于计算/存储费用。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保 Cloud SDK 为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter将以`!`为前缀的行视为shell命令，并以`$`为前缀的Python变量进行插值。

设置您的项目 ID

**如果您不知道您的项目 ID**，您可以使用 `gcloud` 获取您的项目 ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，这将用于这本笔记本其余部分的操作。以下是支持 Vertex AI 的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不能使用多区域存储桶进行 Vertex AI 的训练。并非所有区域都支持所有的 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您参加直播教程会话，您可能正在使用共享测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在**将此服务帐号授予对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入“Vertex”，并选择**Vertex管理员**。在筛选框中输入“Storage Object Admin”，并选择**存储对象管理员**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务帐号密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境如何，以下步骤都是必需的。**

当您初始化Python的Vertex AI SDK时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源相关的所有数据在会话之间保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查Cloud Storage存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量
接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### 初始化顶点 AI SDK 用于 Python

为您的项目和相应的存储桶初始化顶点 AI SDK 用于 Python。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器映像，以及分配给虚拟机实例的 GPU 数量。例如，要使用一个具有 4 个 Nvidia Telsa K80 GPU 的容器映像分配给每个 VM，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定 `(None, None)` 来在 CPU 上运行容器映像。

了解更多关于[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*: 2.3 版本之前的 TF 发行版不支持 GPU，将无法在本教程中加载自定义模型。这是一个已知问题，在 TF 2.3 中已修复。这是由生成在服务函数中的静态图操作引起的。如果您在自己的自定义模型上遇到这个问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建容器

为预测设置预构建的 Docker 容器映像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存。
     - `n1-highcpu`：每个vCPU 0.9GB的内存。
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了来自TensorFlow Hub（TFHub）的预训练模型，然后上传到“Vertex AI Model”资源。一旦有了“Vertex AI Model”资源，这个模型就可以部署到“Vertex AI Endpoint”资源上。

### 下载预训练模型

首先，你需要从TensorFlow Hub下载预训练模型。该模型以TF.Keras层的形式下载。在这个例子中，为了完成模型，你需要创建一个`Sequential()`模型，将下载的TFHub模型作为一个层，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

### 保存模型工件

此时，模型已经存在于内存中。接下来，您可以将模型工件保存到云存储位置。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

## 上传模型用于提供服务

接下来，您将从自定义作业中上传 TF.Keras 模型到 Vertex `Model` 服务，这将为您的自定义模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个提供服务的函数，将数据转换为模型所需的格式。如果您将编码数据发送到 Vertex AI，您的提供服务函数将确保数据在传递到模型之前在模型服务器上进行解码。

### 提供服务函数如何工作

当您向在线预测服务器发送请求时，该请求会被一个 HTTP 服务器接收。HTTP 服务器从 HTTP 请求内容中提取预测请求。提取的预测请求会被转发到提供服务函数。对于 Google 预构建的预测容器，请求内容会被作为 `tf.string` 传递给提供服务函数。

提供服务函数包括两个部分：

- `预处理函数`:
  - 将输入 (`tf.string`) 转换为底层模型的输入形状和数据类型 (动态图)。
  - 执行与在训练底层模型期间相同的数据预处理--例如，归一化、缩放等。
- `后处理函数`:
  - 将模型输出转换为接收应用程序所期望的格式--例如，压缩输出。
  - 为接收应用程序打包输出--例如，添加标题，构建 JSON 对象等。

预处理和后处理函数都被转换为静态图，然后与模型融合在一起。底层模型的输出被传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP 服务器将输出作为 HTTP 响应内容返回。

在构建 TF.Keras 模型的提供服务函数时，您需要考虑一个要点，那就是它们作为静态图运行。这意味着您不能使用需要动态图的 TF 图操作。如果您这样做，您将在提供服务函数的编译过程中遇到错误，并指出您正在使用不受支持的 EagerTensor。

### 图像数据的服务功能

#### 预处理

为了将图像传递给预测服务，您需要将压缩的（例如JPEG）图像字节编码为base 64 -- 这样在通过网络传输二进制数据时可以确保内容不被修改。由于部署的模型期望输入数据为原始（未压缩）字节，您需要确保将base 64编码数据转换回原始字节，然后进行预处理以满足模型输入要求，然后将其作为输入传递给部署的模型。

为了解决这个问题，您需要定义一个服务功能（`serving_fn`）并将其附加到模型作为预处理步骤。添加 `@tf.function` 装饰器以便服务功能与基础模型融合在一起（而不是在CPU上游处理）。

当您发送预测或解释请求时，请求内容将被解码为Tensorflow字符串（`tf.string`），然后传递给服务功能（`serving_fn`）。服务功能将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`），以匹配模型的输入要求：

- `io.decode_jpeg`- 解压JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为浮点32，并重新调整像素数据在0和1之间。
- `image.resize` - 将图像调整大小以匹配模型的输入形状。

此时，数据可以通过具体函数传递给模型（`m_call`）。服务功能是一个静态图，而模型则是一个动态图。具体函数执行从服务功能到模型的输入数据打包的任务，以及将模型的预测结果从模型打包回到服务功能。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(tfhub_model, MODEL_DIR, signatures={"serving_default": serving_fn})

获取服务函数签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要获取服务函数的签名。为什么呢？因为当我们将数据发送给模型进行预测时作为 HTTP 请求包，图像数据是 base64 编码的，而我们的 TF.Keras 模型需要 numpy 输入。您的服务函数将会将 base64 转换为 numpy 数组。

在进行预测请求时，您需要将请求路由到服务函数而不是模型，所以您需要知道服务函数的输入层名称 -- 这将在您之后进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

将TensorFlow Hub模型上传到`Vertex AI Model`资源

最后，将TFHub模型和serving函数中的模型构件上传到`Vertex AI Model`资源中。

*注意：*当您将模型构件上传到`Vertex AI Model`资源时，需指定相应的部署容器镜像。

In [ ]:
model = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

## 创建 `Endpoint` 资源

您可以使用 `Endpoint.create()` 方法创建一个 `Endpoint` 资源。至少，您需要指定端点的显示名称。可选地，您可以指定项目和位置（区域）；否则，设置将继承您在初始化 Vertex AI SDK 时使用 `init()` 方法设置的值。

在这个例子中，指定了以下参数：

- `display_name`: `Endpoint` 资源的可读名称。
- `project`: 您的项目 ID。
- `location`: 您的区域。
- `labels`:（可选）用户定义的元数据，以键/值对的形式存储在 `Endpoint` 中。

此方法返回一个 `Endpoint` 对象。

了解更多关于 [Vertex AI 端点](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="example_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

部署“模型”资源到“端点”资源。

您可以将一个或多个“Vertex AI模型”资源实例部署到同一个端点。每个部署的“Vertex AI模型”资源都将拥有自己的用于提供二进制文件的部署容器。

*注意：* 在这个示例中，您在上传模型工件到“Vertex AI模型”资源的上一步中指定了TFHub模型的部署容器。

在下一个示例中，您将“Vertex AI模型”资源部署到“Vertex AI端点”资源。已经为“Vertex AI模型”资源定义了部署容器图像。要部署，您需要指定以下额外的配置设置：

- 机器类型。
- GPU 的类型和数量（如果有）。
- 静态、手动或自动扩展的 VM 实例。

在这个示例中，您将使用最少数量的指定参数部署模型，如下所示：

- `model`：模型资源。
- `deployed_model_displayed_name`：部署模型实例的人类可读名称。
- `machine_type`：每个 VM 实例的机器类型。

由于需求需要为资源进行配置，这可能需要几分钟时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

### 为预测准备测试数据

接下来，您将把一个压缩的JPEG图片加载到内存中，然后对其进行base64编码。为了演示目的，您将使用来自Flowers数据集的一张图片。

In [ ]:
! gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg test.jpg

In [ ]:
import base64

with open("test.jpg", "rb") as f:
    data = f.read()
b64str = base64.b64encode(data).decode("utf-8")

### 进行预测

现在您的`Model`资源部署到了一个`Endpoint`资源，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 请求

在本示例中，由于您的测试项目位于一个Cloud Storage存储桶中，您可以使用`tf.io.gfile.Gfile()`来打开并读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为base64 -- 这样可以在通过网络传输二进制数据时保证内容不被篡改。

每个实例的格式为：

    { serving_input: { 'b64': base64编码的字节 } }

由于`predict()`方法可以接受多个项目（实例），请将您的单个测试项目作为包含一个测试项目的列表发送。

#### 响应

从`predict()`调用返回的是一个Python字典，包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个类别标签的预测置信度，范围在0到1之间。
- `deployed_model_id`：执行预测的Vertex AI中`Model`资源的标识符。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: {"b64": b64str}}]

prediction = endpoint.predict(instances=instances)

print(prediction)

清理

要清理该项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_endpoint = True

if delete_endpoint:
    try:
        endpoint.undeploy_all()
        endpoint.delete()
    except Exception as e:
        print(e)

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}